In [10]:
from db import Eleicao, Estado, Municipio, Zona, Secao, Log, MunicipioZona
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import datetime as dt
from ipywidgets import widgets
from ipywidgets import interact, interact_manual, interactive_output, VBox, HBox, Accordion, Box, Layout

sns.set_palette("bright")

# Plot Function
def plot_graph(eleicao=1, estado=1, municipio=1, zona=1, secao=1,tipo= 'Histograma', x='tipo' , y='urna', color_x='blue'):
    if(not eleicao or not estado or not municipio or not zona or not secao or not tipo or not x):
        return 
    else:
        global eleicoes, estados, municipios, zonas, secoes, opcoes
        plt.figure(dpi=120)
        sns.set_style('whitegrid')
        mes = 10
        if(eleicao == 1):
            dia = 2
        else:
            dia = 30

        query = Log.select().where((Log.secao == secao) & (Log.datetime.day == dia) & (Log.datetime.month == mes) )
        df = pd.DataFrame(list(query.dicts()))

        #quantidade de votos
        votos = df[df.descricao == 'O voto do eleitor foi computado']
        votos_d.value = str(votos.shape[0])
        
        #comeco e fim das atividades
        inicio = df.iloc[0]
        fim = df.iloc[-1]

        inicio_d.value = str(inicio['datetime'].time())
        fim_d.value = str(fim['datetime'].time())

        #primeiro e ultimo votos
        primeiro = votos.iloc[0]
        ultimo = votos.iloc[-1]

        primeiro_d.value = str(primeiro['datetime'].time())
        ultimo_d.value = str(ultimo['datetime'].time())

        #média de votos por hora
        #horas
        horas = pd.to_datetime(df.datetime).dt.hour.unique()

        #resultados
        result = pd.DataFrame(data=horas, columns=['Horas'])


        # Dividir por hora
        logs_por_hora = [g.reset_index(drop=True) for i,g in df.groupby(pd.to_datetime(df.datetime).dt.hour)]

        cont = 0
        votos = []

        for dy in logs_por_hora:
            voto = dy[dy.descricao == 'O voto do eleitor foi computado'].shape[0]
            votos.append(voto)
            cont += 1

        result['Votos'] = votos
        result['Porcentagem'] = result['Votos'] / result['Votos'].sum()

        
        votos_hora = result[result['Votos'] != 0]

        media_votos_hora_d.value = str(round(votos_hora['Votos'].mean(),2))

        #tempo médio de votação
        def format(tdelta):
            horas, rem =  divmod(tdelta.seconds, 3600)
            minutos, segundos = divmod(rem, 60)
            time = '{:02d}:{:02d}:{:02d}'.format(horas,minutos, segundos)
            return time
        

        comeco = df.reset_index().index[df.descricao == 'Título digitado pelo mesário'].tolist()
        fim = df.reset_index().index[df.descricao == 'O voto do eleitor foi computado'].tolist()
        comeco.reverse()
        fim.reverse()

        xs = []
        temp = []
        for b in fim:
            for a in comeco:
                if(a < b):
                    xs.append(a)
                    temp.append((a,b))
                    break

        temp.reverse()
        comeco.reverse()
        fim.reverse()
        xs.reverse()
        temp2 = [c for c in comeco if c not in xs]


        tempos = []
        eleitores = [df.iloc[temp[d][0]:temp[d][1]] for d in range(0, len(temp)) ]
        for eleitor in eleitores:
            inicio = eleitor.iloc[0]['datetime']
            fim = eleitor.iloc[-1]['datetime']
            diff = (fim - inicio)
            tempos.append(diff)
            # tempos.append(pd.to_datetime().dt.time)

        temp_votacao  = pd.DataFrame(tempos, columns=['Tempo em Votação'])
        tempo_medio_vota_d.value = format(temp_votacao['Tempo em Votação'].mean())

        #tempo médio na urna
        
        comeco = df.reset_index().index[df.descricao == 'Eleitor foi habilitado'].tolist()
        fim = df.reset_index().index[df.descricao == 'O voto do eleitor foi computado'].tolist()
        comeco.reverse()
        fim.reverse()

        xs = []
        temp = []
        for b in fim:
            for a in comeco:
                if(a < b):
                    xs.append(a)
                    temp.append((a,b))
                    break

        temp.reverse()
        comeco.reverse()
        fim.reverse()
        xs.reverse()


        tempos = []
        eleitores = [df.iloc[temp[d][0]:temp[d][1]] for d in range(0, len(temp)) ]
        for eleitor in eleitores:
            inicio = eleitor.iloc[0]['datetime']
            fim = eleitor.iloc[-1]['datetime']
            diff = (fim - inicio)
            tempos.append(diff)
            

        temp_urna  = pd.DataFrame(tempos, columns=['Tempo na Urna'])

        tempo_medio_na_urna_d.value = format(temp_urna['Tempo na Urna'].mean())

        #dificuldade na votação

        comeco = df.reset_index().index[df.descricao == 'Eleitor foi habilitado'].tolist()
        fim = df.reset_index().index[df.descricao == 'O voto do eleitor foi computado'].tolist()

        comeco.reverse()
        fim.reverse()

        xs = []
        temp = []
        for n in fim:
            for m in comeco:
                if(m < n):
                    xs.append(m)
                    temp.append((m,n))
                    break

        temp.reverse()
        comeco.reverse()
        fim.reverse()
        xs.reverse()

        erros = []
        eleitores = [df.iloc[temp[x][0]:temp[x][1]] for x in range(0, len(temp)) ]
        for eleitor in eleitores:
            erros_durante_votacao = eleitor[eleitor.descricao == 'Tecla indevida pressionada'].shape[0]
            erros.append(erros_durante_votacao)

        votacao = pd.DataFrame(erros, columns=['Erros na Votação'])

        #dificuldade no reconhecimento da digital
        comeco = df.reset_index().index[df.descricao == 'Título digitado pelo mesário'].tolist()
        fim = df.reset_index().index[df.descricao == 'O voto do eleitor foi computado'].tolist()
        comeco.reverse()
        fim.reverse()

        xs = []
        temp = []
        for n in fim:
            for m in comeco:
                if(m < n):
                    xs.append(m)
                    temp.append((m,n))
                    break

        temp.reverse()
        comeco.reverse()
        fim.reverse()
        xs.reverse()


        erros_digitais = []
        eleitores = [df.iloc[temp[x][0]:temp[x][1]] for x in range(0, len(temp)) ]
        for eleitor in eleitores:
            erro_digital = eleitor[eleitor['descricao'].str.contains('Digital capturada não corresponde a digital do eleitor')].shape[0]
            erros_digitais.append(erro_digital)

        digital  = pd.DataFrame(erros_digitais, columns=['Erros na Digital'])
        
        #agregando em um único dataset

        database = temp_votacao.join(temp_urna).join(digital).join(votacao)


        if(tipo == 'Histograma'):
            if(x == 'Votos por Hora'):
                ax = result.plot(kind='bar', x = 'Horas', y='Votos', color=color_x)

                for p in ax.patches:
                    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

                return ax
            elif(x == 'Porcentagem por Hora'):
                
                ax = result.plot(kind='bar', x = 'Horas', y='Porcentagem', color=color_x)

                for p in ax.patches:
                    ax.annotate(str(round(p.get_height() * 100, 2))+'%', (p.get_x() * 1.005, p.get_height() * 1.005))

                return ax
            elif(x =='Tempo em Votação'):
                temp3 = temp_votacao.astype('timedelta64[s]') / 60
                ax2 = temp3.plot.hist(color=color_x)
                for p in ax2.patches:
                    ax2.annotate(str(int(p.get_height())), (p.get_x() * 1.01, p.get_height() * 1.005))
                return ax2

            elif(x =='Tempo na Urna'):
                temp = temp_urna.astype('timedelta64[s]') / 60
                ax3 = temp.plot.hist(color=color_x)
                for p in ax3.patches:
                    ax3.annotate(str(int(p.get_height())), (p.get_x() * 1.01, p.get_height() * 1.005))

                return ax3

            elif(x =='Erros na Digital'):
                ax4 = digital.plot.hist(color = color_x, )
                ax4.locator_params(axis='x', integer=True)
                for p in ax4.patches:
                    ax4.annotate(str(int(p.get_height())), (p.get_x() * 1.005, p.get_height() * 1.005))
                return ax4

            elif(x =='Erros na Votação'):
                ax5 = votacao.plot.hist(color=color_x)
                for p in ax5.patches:
                    ax5.annotate(str(int(p.get_height())), (p.get_x() * 1.01, p.get_height() * 1.005))

                return ax5

            else:
                return 
            
        elif(tipo == 'Dispersão'):
            return sns.scatterplot(data=database, x=database[x], y=database[y], alpha=0.9)
        
        elif(tipo == 'Calor'):
           
            if(x == 'Votos por Hora'):
                df = result[['Horas', 'Votos']]
                df = df.set_index('Horas')

                ax = sns.heatmap(df, annot=True, cmap="crest")

                return ax.invert_yaxis()

                
            
            elif(x == 'Porcentagem por Hora'):
                
                df = result[['Horas', 'Porcentagem']]
                df = df.set_index('Horas')

                ax = sns.heatmap(df, annot=True, cmap="crest")

                return ax.invert_yaxis()

            
        #liberar grafico
       
#Functions

def observe_eleicao(_):
    if(eleicao_d.value >= 1):
        query = Estado.select().where(Estado.eleicao == eleicao_d.value)
        b = list(query.dicts())
        def get_list_estado(esta):
            return (esta['nome'], esta['id'])
        estados = [('Selecione o Estado...', 0)] +list(map(get_list_estado, b))
        estado_d.options = estados
        estado_d.disabled = False

def observe_estado(_):
    if(estado_d.value >= 1):
        query = Municipio.select().where(Municipio.estado == estado_d.value)
        c = list(query.dicts())
        def get_list_municipio(muni):
            return (muni['nome'], muni['id'])
        municipios = [('Selecione o Municipio...', 0)] + list(map(get_list_municipio, c))
        municipio_d.options = municipios
        municipio_d.disabled = False
    
def observe_municipio(_):
    if(municipio_d.value >= 1):
        query = Zona.select().join(MunicipioZona).where(MunicipioZona.municipio == municipio_d.value)
        d = list(query.dicts())
        def get_list_zona(zon):
            return (zon['nome'], zon['id'])
        zonas = [('Selecione a Zona...', 0)] + list(map(get_list_zona, d))
        zona_d.options = zonas
        zona_d.disabled = False

def observe_zona(_):
    if(zona_d.value >= 1 & municipio_d.value >= 1):
        query = Secao.select().where((Secao.municipio == municipio_d.value) & (Secao.zona == zona_d.value))
        e = list(query.dicts())
        def get_list_secao(sec):
            return (sec['nome'], sec['id'])
        secoes = [('Selecione a Seção...', 0)] + list(map(get_list_secao, e))
        secao_d.options = secoes
        secao_d.disabled = False

def observe_secao(_):
    if(secao_d.value >= 0):
        tipo_d.disabled = False

def observe_tipo(_):
    x_d.disabled = False
    if(tipo_d.value == 'Histograma'):
        x_d.options = opcoes2
        y_d.disabled = True
    elif():
        x_d.options = opcoes3
        y_d.disabled = True
    else:
        x_d.options = opcoes
        y_d.disabled = False


#Default
query = Eleicao.select()
a = list(query.dicts())
def get_list_eleicao(eleicao):
    return ('{} - {}º turno'.format(eleicao['ano'],eleicao['turno']), eleicao['id'])
eleicoes = [('Selecione a Eleição...', 0)] + list(map(get_list_eleicao, a))
estados = [('Selecione o Estado...', 0)]
municipios = [('Selecione o Municipio...', 0)]
zonas = [('Selecione a Zona...', 0)]
secoes = [('Selecione a Seção...', 0)]
opcoes = ['Tempo em Votação', 'Tempo na Urna', 'Erros na Digital', 'Erros na Votação']
opcoes2 = ['Votos por Hora', 'Porcentagem por Hora',  'Tempo em Votação', 'Tempo na Urna', 'Erros na Digital', 'Erros na Votação']
opcoes3 = ['Votos por Hora', 'Porcentagem por Hora']
tipos = ['Histograma', 'Dispersão', 'Calor']

style = {'description_width': 'initial'}

#Entradas
eleicao_d = widgets.Dropdown(
    options = eleicoes,
    value = eleicoes[0][1],
    description = 'Eleição'
)
estado_d = widgets.Dropdown(
    options = estados,
    value = estados[0][1],
    description = "Estado",
    disabled = True
)

municipio_d = widgets.Dropdown(
    options = municipios,
    value = municipios[0][1],
    description = "Municipio",
    disabled = True
)
zona_d = widgets.Dropdown(
    options = zonas,
    value = zonas[0][1],
    description = "Zona",
    disabled = True
)
secao_d = widgets.Dropdown(
    options = secoes,
    value = secoes[0][1],
    description = "Seção",
    disabled = True
)
tipo_d = widgets.Dropdown(
    options = tipos,
    value = tipos[0],
    description = "Tipo",
    disabled = True
)
x_d = widgets.Dropdown(
    options = opcoes,
    value = opcoes[0],
    description = "X",
    disabled = True
)
y_d = widgets.Dropdown(
    options = opcoes,
    value = opcoes[1],
    description = "Y",
    disabled = True
)
color_xd = widgets.ColorPicker(
    concise=False,
    description='Cor',
    value='#7eb0d5',
    disabled=False
)
# color_yd = widgets.ColorPicker(
#     concise=False,
#     description='Cor Y',
#     value='#b2e061',
#     disabled=False
# )


#Dados
votos_d = widgets.Text(
    value = '',
    description = "Votos Computados",
    disabled = True,
    style=style
)

inicio_d = widgets.Text(
    value = '',
    description = "Inicio das Atividades",
    disabled = True,
    style=style
)

fim_d = widgets.Text(
    value = '',
    description = "Fim das Atividades",
    disabled = True,
    style=style
)

primeiro_d = widgets.Text(
    value = '',
    description = "Primeiro Voto da Seção",
    disabled = True,
    style=style
)

ultimo_d = widgets.Text(
    value = '',
    description = "Último Voto da Seção",
    disabled = True,
    style=style
)

media_votos_hora_d = widgets.Text(
    value = '',
    description = "Média de Votos por Hora",
    disabled = True,
    style=style
)

tempo_medio_vota_d = widgets.Text(
    value = '',
    description = "Tempo Médio de Votação",
    disabled = True,
    style=style
)

tempo_medio_na_urna_d = widgets.Text(
    value = '',
    description = "Tempo Médio na Urna",
    disabled = True,
    style=style
)




#Observe
eleicao_d.observe(observe_eleicao)
estado_d.observe(observe_estado)
municipio_d.observe(observe_municipio)
zona_d.observe(observe_zona)
secao_d.observe(observe_secao)
tipo_d.observe(observe_tipo)

out = interactive_output(plot_graph,{
    "eleicao" : eleicao_d,
    "estado" : estado_d,
    "municipio" : municipio_d,
    "zona" : zona_d,
    "secao" : secao_d,
    "tipo": tipo_d,
    "x" : x_d,
    "y" : y_d,
    "color_x": color_xd
    },
       
)

#Layout
ui = VBox([eleicao_d, estado_d,municipio_d, zona_d, secao_d, tipo_d, x_d, y_d,color_xd], layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='center',
    justify_content = 'space-around',
    flex = '1 1 auto',
    width = 'auto'
))

box2 = HBox([out], layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid red 2px',
    align_items='center',
    justify_content = 'space-around',
    flex = '3 1 auto',
    width = 'auto'
))
data = HBox([votos_d, inicio_d, fim_d, primeiro_d, ultimo_d, media_votos_hora_d, tempo_medio_vota_d, tempo_medio_na_urna_d], layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid  2px',
    align_items='center',
    justify_content = 'space-around',
    flex = '1 1 auto',
    width = 'auto'
))
box = HBox([ui, box2 , data], layout=Layout(
    display='flex',
    flex_flow='row',
    align_items='stretch',
    justify_content = 'space-around', 
))


display(box)

